In [4]:
# base packages to work with AMLS
import azureml.core
from azureml.core import Workspace
print("Azure ML version:", azureml.core.VERSION)

'1.7.0'

### Initialize the Workspace

In [5]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config() # Currently the config.json is not provided for security purposes.
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

grab	southeastasia	main-resource	southeastasia


### Initialize an Experiment

In [6]:
# Name your experiment here.
experiment_name = 'grab-exp'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Attach a Compute Resource

In [7]:
# Compute cluster creation.
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "ml-compute-2"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    #compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
    #                                                       max_nodes=4)
    #cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    print("Cluster does not exists.")

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Environment Set Up

In [19]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

fwrk = Environment("grab-env")

fwrk.docker.enabled = True
fwrk.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas',
                                                                          'pyarrow',
                                                                          'lightgbm',
                                                                          'matplotlib',
                                                                          'scikit-learn'
                                                                         ])
fwrk.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
fwrk.python.conda_dependencies.save_to_file(".", "grab-conda.yml")

# Register environment to re-use later
fwrk.register(workspace=ws)

'grab-conda.yml'

### Register Dataset

In [11]:
from azureml.core.dataset import Dataset

web_paths = [
            'https://grab5033896937.blob.core.windows.net/azureml/Dataset/grab/part-00000-41e3fe2a-7fad-41df-aba6-d805d478bc9f-c000.snappy.parquet'
            ]
dataset = Dataset.File.from_files(path=web_paths)

dataset = dataset.register(workspace=ws,
                           name='grab_dataset',
                           description='training and test dataset',
                           create_new_version=True)

### Submit the Training Job to the Compute Cluster

In [14]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

script_folder = "train"
src = ScriptRunConfig(source_directory=script_folder, script='train.py')

# Set compute target to the one created in previous step
src.run_config.target = cpu_cluster.name

# Set environment
src.run_config.environment = fwrk
 
run = exp.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
grab-exp,grab-exp_1592730726_88b5cd0a,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


### Watch the execution of the Training

In [15]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [16]:
%%time
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True) 

RunId: grab-exp_1592730726_88b5cd0a
Web View: https://ml.azure.com/experiments/grab-exp/runs/grab-exp_1592730726_88b5cd0a?wsid=/subscriptions/c681c312-5132-4e43-a366-0911473c2107/resourcegroups/main-resource/workspaces/grab

Streaming azureml-logs/55_azureml-execution-tvmps_6c1ac0ee6fae8abfbaef68f2cb1d2d90ba51f4b4201acb9dd58037c9672d3421_d.txt

Using default tag: latest
latest: Pulling from azureml/azureml_a683d2179a44e45edf6b076dc7e167da
Digest: sha256:1b18fdb35b0c24e2507ef85cec951998a02aac4b096782d65a61d1f53d63ed5a
Status: Image is up to date for grabddc843c6.azurecr.io/azureml/azureml_a683d2179a44e45edf6b076dc7e167da:latest
2020-06-21T09:12:44Z Starting output-watcher...
2020-06-21T09:12:44Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
6bd271419afdd8738327c6d321e3011db11d811f8b79872e37e95f587edc10a7

Streaming azureml-logs/65_job_prep-tvmps_6c1ac0ee6fae8abfbaef68f2cb1d2d90ba51f4b4201acb9dd58037c9672d3421_d.txt

Entering job preparation. Current time:2020-06-21T09:12

[981]	valid_0's rmse: 208.688
[982]	valid_0's rmse: 208.643
[983]	valid_0's rmse: 208.613
[984]	valid_0's rmse: 208.568
[985]	valid_0's rmse: 208.538
[986]	valid_0's rmse: 208.496
[987]	valid_0's rmse: 208.457
[988]	valid_0's rmse: 208.414
[989]	valid_0's rmse: 208.37
[990]	valid_0's rmse: 208.331
[991]	valid_0's rmse: 208.293
[992]	valid_0's rmse: 208.254
[993]	valid_0's rmse: 208.216
[994]	valid_0's rmse: 208.178
[995]	valid_0's rmse: 208.135
[996]	valid_0's rmse: 208.092
[997]	valid_0's rmse: 208.048
[998]	valid_0's rmse: 208.007
[999]	valid_0's rmse: 207.969
[1000]	valid_0's rmse: 207.932
[1001]	valid_0's rmse: 207.889
[1002]	valid_0's rmse: 207.853
[1003]	valid_0's rmse: 207.81
[1004]	valid_0's rmse: 207.766
[1005]	valid_0's rmse: 207.726
[1006]	valid_0's rmse: 207.684
[1007]	valid_0's rmse: 207.641
[1008]	valid_0's rmse: 207.601
[1009]	valid_0's rmse: 207.566
[1010]	valid_0's rmse: 207.523
[1011]	valid_0's rmse: 207.487
[1012]	valid_0's rmse: 207.451
[1013]	valid_0's rmse: 207.40

[2362]	valid_0's rmse: 184.533
[2363]	valid_0's rmse: 184.526
[2364]	valid_0's rmse: 184.519
[2365]	valid_0's rmse: 184.514
[2366]	valid_0's rmse: 184.506
[2367]	valid_0's rmse: 184.498
[2368]	valid_0's rmse: 184.49
[2369]	valid_0's rmse: 184.482
[2370]	valid_0's rmse: 184.474
[2371]	valid_0's rmse: 184.468
[2372]	valid_0's rmse: 184.461
[2373]	valid_0's rmse: 184.453
[2374]	valid_0's rmse: 184.444
[2375]	valid_0's rmse: 184.436
[2376]	valid_0's rmse: 184.429
[2377]	valid_0's rmse: 184.422
[2378]	valid_0's rmse: 184.414
[2379]	valid_0's rmse: 184.408
[2380]	valid_0's rmse: 184.401
[2381]	valid_0's rmse: 184.392
[2382]	valid_0's rmse: 184.384
[2383]	valid_0's rmse: 184.378
[2384]	valid_0's rmse: 184.373
[2385]	valid_0's rmse: 184.365
[2386]	valid_0's rmse: 184.358
[2387]	valid_0's rmse: 184.353
[2388]	valid_0's rmse: 184.348
[2389]	valid_0's rmse: 184.341
[2390]	valid_0's rmse: 184.334
[2391]	valid_0's rmse: 184.327
[2392]	valid_0's rmse: 184.32
[2393]	valid_0's rmse: 184.313
[2394]	val

[3829]	valid_0's rmse: 179.157
[3830]	valid_0's rmse: 179.152
[3831]	valid_0's rmse: 179.15
[3832]	valid_0's rmse: 179.149
[3833]	valid_0's rmse: 179.146
[3834]	valid_0's rmse: 179.145
[3835]	valid_0's rmse: 179.142
[3836]	valid_0's rmse: 179.14
[3837]	valid_0's rmse: 179.138
[3838]	valid_0's rmse: 179.136
[3839]	valid_0's rmse: 179.136
[3840]	valid_0's rmse: 179.133
[3841]	valid_0's rmse: 179.131
[3842]	valid_0's rmse: 179.13
[3843]	valid_0's rmse: 179.127
[3844]	valid_0's rmse: 179.126
[3845]	valid_0's rmse: 179.124
[3846]	valid_0's rmse: 179.121
[3847]	valid_0's rmse: 179.12
[3848]	valid_0's rmse: 179.118
[3849]	valid_0's rmse: 179.116
[3850]	valid_0's rmse: 179.114
[3851]	valid_0's rmse: 179.113
[3852]	valid_0's rmse: 179.111
[3853]	valid_0's rmse: 179.109
[3854]	valid_0's rmse: 179.107
[3855]	valid_0's rmse: 179.105
[3856]	valid_0's rmse: 179.104
[3857]	valid_0's rmse: 179.102
[3858]	valid_0's rmse: 179.1
[3859]	valid_0's rmse: 179.098
[3860]	valid_0's rmse: 179.096
[3861]	valid_0

[6961]	valid_0's rmse: 174.106
[6962]	valid_0's rmse: 174.106
[6963]	valid_0's rmse: 174.105
[6964]	valid_0's rmse: 174.103
[6965]	valid_0's rmse: 174.102
[6966]	valid_0's rmse: 174.1
[6967]	valid_0's rmse: 174.098
[6968]	valid_0's rmse: 174.097
[6969]	valid_0's rmse: 174.097
[6970]	valid_0's rmse: 174.096
[6971]	valid_0's rmse: 174.095
[6972]	valid_0's rmse: 174.094
[6973]	valid_0's rmse: 174.093
[6974]	valid_0's rmse: 174.092
[6975]	valid_0's rmse: 174.091
[6976]	valid_0's rmse: 174.09
[6977]	valid_0's rmse: 174.089
[6978]	valid_0's rmse: 174.088
[6979]	valid_0's rmse: 174.087
[6980]	valid_0's rmse: 174.086
[6981]	valid_0's rmse: 174.085
[6982]	valid_0's rmse: 174.084
[6983]	valid_0's rmse: 174.083
[6984]	valid_0's rmse: 174.081
[6985]	valid_0's rmse: 174.079
[6986]	valid_0's rmse: 174.078
[6987]	valid_0's rmse: 174.076
[6988]	valid_0's rmse: 174.074
[6989]	valid_0's rmse: 174.073
[6990]	valid_0's rmse: 174.072
[6991]	valid_0's rmse: 174.071
[6992]	valid_0's rmse: 174.07
[6993]	valid

...

[30720]	valid_0's rmse: 167.052
[30721]	valid_0's rmse: 167.052
[30722]	valid_0's rmse: 167.051
[30723]	valid_0's rmse: 167.05
[30724]	valid_0's rmse: 167.05
[30725]	valid_0's rmse: 167.05
[30726]	valid_0's rmse: 167.049
[30727]	valid_0's rmse: 167.049
[30728]	valid_0's rmse: 167.048
[30729]	valid_0's rmse: 167.048
[30730]	valid_0's rmse: 167.048
[30731]	valid_0's rmse: 167.048
[30732]	valid_0's rmse: 167.048
[30733]	valid_0's rmse: 167.048
[30734]	valid_0's rmse: 167.048
[30735]	valid_0's rmse: 167.048
[30736]	valid_0's rmse: 167.048
[30737]	valid_0's rmse: 167.048
[30738]	valid_0's rmse: 167.048
[30739]	valid_0's rmse: 167.048
[30740]	valid_0's rmse: 167.048
[30741]	valid_0's rmse: 167.048
[30742]	valid_0's rmse: 167.047
[30743]	valid_0's rmse: 167.047
[30744]	valid_0's rmse: 167.047
[30745]	valid_0's rmse: 167.046
[30746]	valid_0's rmse: 167.047
[30747]	valid_0's rmse: 167.046
[30748]	valid_0's rmse: 167.046
[30749]	valid_0's rmse: 167.046
[30750]	valid_0's rmse: 167.045
[30751]	val

[34714]	valid_0's rmse: 166.789
[34715]	valid_0's rmse: 166.788
[34716]	valid_0's rmse: 166.789
[34717]	valid_0's rmse: 166.789
[34718]	valid_0's rmse: 166.789
[34719]	valid_0's rmse: 166.789
[34720]	valid_0's rmse: 166.788
[34721]	valid_0's rmse: 166.788
[34722]	valid_0's rmse: 166.788
[34723]	valid_0's rmse: 166.788
[34724]	valid_0's rmse: 166.788
[34725]	valid_0's rmse: 166.788
[34726]	valid_0's rmse: 166.788
[34727]	valid_0's rmse: 166.787
[34728]	valid_0's rmse: 166.787
[34729]	valid_0's rmse: 166.787
[34730]	valid_0's rmse: 166.787
[34731]	valid_0's rmse: 166.787
[34732]	valid_0's rmse: 166.786
[34733]	valid_0's rmse: 166.786
[34734]	valid_0's rmse: 166.786
[34735]	valid_0's rmse: 166.785
[34736]	valid_0's rmse: 166.785
[34737]	valid_0's rmse: 166.784
[34738]	valid_0's rmse: 166.784
[34739]	valid_0's rmse: 166.784
[34740]	valid_0's rmse: 166.784
[34741]	valid_0's rmse: 166.784
[34742]	valid_0's rmse: 166.784
[34743]	valid_0's rmse: 166.784
[34744]	valid_0's rmse: 166.784
[34745]	

[39012]	valid_0's rmse: 166.572
[39013]	valid_0's rmse: 166.571
[39014]	valid_0's rmse: 166.571
[39015]	valid_0's rmse: 166.57
[39016]	valid_0's rmse: 166.57
[39017]	valid_0's rmse: 166.57
[39018]	valid_0's rmse: 166.569
[39019]	valid_0's rmse: 166.569
[39020]	valid_0's rmse: 166.569
[39021]	valid_0's rmse: 166.569
[39022]	valid_0's rmse: 166.569
[39023]	valid_0's rmse: 166.569
[39024]	valid_0's rmse: 166.569
[39025]	valid_0's rmse: 166.569
[39026]	valid_0's rmse: 166.569
[39027]	valid_0's rmse: 166.569
[39028]	valid_0's rmse: 166.569
[39029]	valid_0's rmse: 166.569
[39030]	valid_0's rmse: 166.569
[39031]	valid_0's rmse: 166.57
[39032]	valid_0's rmse: 166.569
[39033]	valid_0's rmse: 166.569
[39034]	valid_0's rmse: 166.569
[39035]	valid_0's rmse: 166.569
[39036]	valid_0's rmse: 166.569
[39037]	valid_0's rmse: 166.568
[39038]	valid_0's rmse: 166.568
[39039]	valid_0's rmse: 166.568
[39040]	valid_0's rmse: 166.568
[39041]	valid_0's rmse: 166.568
[39042]	valid_0's rmse: 166.568
[39043]	vali


Streaming azureml-logs/75_job_post-tvmps_6c1ac0ee6fae8abfbaef68f2cb1d2d90ba51f4b4201acb9dd58037c9672d3421_d.txt

Entering job release. Current time:2020-06-21T09:16:15.493604
Starting job release. Current time:2020-06-21T09:16:16.288177
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 333
Entering context manager injector. Current time:2020-06-21T09:16:16.307143
Job release is complete. Current time:2020-06-21T09:16:18.380972

Execution Summary
RunId: grab-exp_1592730726_88b5cd0a
Web View: https://ml.azure.com/experiments/grab-exp/runs/grab-exp_1592730726_88b5cd0a?wsid=/subscriptions/c681c312-5132-4e43-a366-0911473c2107/resourcegroups/main-resource/workspaces/grab

Wall time: 5min


{'runId': 'grab-exp_1592730726_88b5cd0a',
 'target': 'ml-compute-2',
 'status': 'Completed',
 'startTimeUtc': '2020-06-21T09:12:29.642936Z',
 'endTimeUtc': '2020-06-21T09:16:28.037823Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'c4212f64-2fa5-4b1e-b43d-da24cc61927b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'ml-compute-2',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'grab-env',
   'version': 'Autosave_2020-06-20T19:52:37Z_d992251a',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'channels': ['anacond

### Register the Model

In [17]:
# register model 
model = run.register_model(model_name='grab-model', model_path='outputs/lightgbm.pkl')
print(model.name, model.id, model.version, sep='\t')

from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config()
model = Model(ws, 'grab-model')

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "lightgbm.pkl")

os.stat(file_path)

grab-model	grab-model:1	1


os.stat_result(st_mode=33206, st_ino=2251799813784154, st_dev=1154982114, st_nlink=1, st_uid=0, st_gid=0, st_size=176690063, st_atime=1592670451, st_mtime=1592731061, st_ctime=1592670425)

### Deploy the Model

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

# Initialise the workspace
ws = Workspace.from_config()
model = Model(ws, 'grab-model')

# Create environment file 
#myenv = Environment.get(workspace=ws, name="grab-env") #version="1"
inference_config = InferenceConfig(entry_script="score.py", environment=fwrk) #myenv

# Create configuration file 
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

# Deploy the model
service = Model.deploy(workspace=ws, 
                       name='grab-eta2', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

In [ ]:
service.get_logs()